***GENERATED CODE FOR crashesclassify PIPELINE.***

***DON'T EDIT THIS CODE.***

***CONNECTOR FUNCTIONS TO READ DATA.***

In [ ]:
import os
import datetime
import logging
import warnings
warnings.filterwarnings('ignore')
logging.basicConfig(format='%(levelname)s:%(message)s', level=logging.INFO)


class HDFSConnector:

    def fetch(spark, config):
        ################### INPUT HADOOP HOST PORT TO CONNECT WITH ###############################
        hdfs_server = str(os.environ['HDFS_SERVER'])
        hdfs_port = int(os.environ['HDFS_PORT'])
        df = spark.read.options(header='true', inferschema='true').csv(
            f"hdfs://{hdfs_server}:{hdfs_port}{eval(config)['url']}", header='true')
        display(df.limit(2).toPandas())
        return df

    def put(df, spark, config):
        return df.write.format('csv').options(header='true' if eval(config)["is_header"] == "Use Header Line" else 'false',
                                              delimiter=eval(config)["delimiter"]).save(("%s %s") % (datetime.datetime.now().strftime("%Y-%m-%d %H.%M.%S")+"_", eval(config)['url']))


***TRANSFORMATIONS FUNCTIONS THAT WILL BE APPLIED ON DATA***

In [ ]:
import json
from pyspark.sql.types import IntegerType
from pyspark.ml.feature import StringIndexer
from pyspark.sql.functions import col, when
from pyspark.sql.types import IntegerType
from pyspark.sql.functions import mean, stddev, min, max, col


class CleanseData:
    # def __init__(self,df):
    #     #print()

    def cleanValueForFE(self, value):
        if value == None:
            return ""
        elif str(value) == 'nan':
            return "nan"
        else:
            return value

    def replaceByMean(self, feature, df, mean_=-1):
        df1 = df
        df1 = df1.dropna()
        meanValue = self.cleanValueForFE(df1.select(
            mean(col(feature.name)).alias('mean')).collect()[0]["mean"])
        df = df.fillna(meanValue, subset=[feature.name])
        df.withColumn(feature.name, when(col(feature.name) == " ",
                      meanValue).otherwise(col(feature.name).cast("Integer")))
        return df

    def replaceByMax(self, feature, df, max_=-1):
        df1 = df
        df1 = df1.dropna()
        maxValue = self.cleanValueForFE(df1.select(
            max(col(feature.name)).alias('max')).collect()[0]["max"])
        df = df.fillna(maxValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", maxValue).otherwise(col(feature.name)))
        return df

    def replaceByMin(self, feature, df, min_=-1):
        df1 = df
        df1 = df1.dropna()
        minValue = self.cleanValueForFE(df1.select(
            min(col(feature.name)).alias('min')).collect()[0]["min"])
        df = df.fillna(minValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", minValue).otherwise(col(feature.name)))
        return df

    def replaceByStandardDeviation(self, feature, df, stddev_=-1):
        df1 = df
        df1 = df1.dropna()
        stddevValue = self.cleanValueForFE(df1.select(
            stddev(col(feature.name)).alias('stddev')).collect()[0]["stddev"])
        df = df.fillna(stddevValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", stddevValue).otherwise(col(feature.name)))
        return df

    def replaceDateRandomly(self, feature, df):
        df1 = df
        df1 = df1.dropna()
        fillValue = self.cleanValueForFE(
            df.where(col(feature.name).isNotNull()).head(1)[0][feature.name])
        df = df.fillna(str(fillValue), subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", fillValue).otherwise(col(feature.name)))
        # print("CleanseData:replaceDateRandomly Schema : ", df.#printSchema())
        return df

    def replaceNullValues(self, fList, df):
        featuresList = df.schema.fields
        for featureObj in fList:
            for feat in featuresList:
                if featureObj["feature"] in feat.name:
                    featureName = feat
                    if "mean" in featureObj["replaceby"]:
                        df = self.replaceByMean(featureName, df)
                    elif "max" in featureObj["replaceby"]:
                        df = self.replaceByMax(featureName, df)
                    elif "min" in featureObj["replaceby"]:
                        df = self.replaceByMin(featureName, df)
                    elif "stddev" in featureObj["replaceby"]:
                        df = self.replaceByStandardDeviation(featureName, df)
                    elif "random" in featureObj["replaceby"]:
                        df = self.replaceDateRandomly(featureName, df)
        return df


def StringIndexerTransform(df, params, transformationData={}):
    dfReturn = df
    feature = params["feature"]

    dfReturn = dfReturn.fillna({feature: ''})
    outcol = feature + "_stringindexer"
    indexer = StringIndexer(
        inputCol=feature, outputCol=outcol, handleInvalid="skip")
    indexed = indexer.fit(dfReturn).transform(dfReturn)
    dfReturn = indexed
    distinct_values_list = dfReturn.select(
        outcol).distinct().rdd.map(lambda r: r[0]).collect()
    len_distinct_values_list = len(distinct_values_list)
    if len_distinct_values_list <= 4:
        changed_type_df = dfReturn.withColumn(
            outcol, dfReturn[outcol].cast(IntegerType()))
        return changed_type_df
    return dfReturn


class TransformationMain:
    # TODO: change df argument in run with following
    def run(transformationDF, config):
        configObj = json.loads(config)
        featureData = configObj["FE"]
        transformationDF = CleanseData().replaceNullValues(featureData, transformationDF)
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Date', 'transformation_label': 'String Indexer'}], 'feature': 'Date', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
                                                  'count': '500', 'mean': '', 'stddev': '', 'min': '1/11/1992', 'max': '9/9/1991', 'missing': '0', 'distinct': '414'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Date'}, {'feature_label': 'Date', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Date')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Aircraft', 'transformation_label': 'String Indexer'}], 'feature': 'Aircraft', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': 'Aeritalia G.222', 'max': 'Yakovlev Yak-42', 'missing': '0', 'distinct': '162'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Aircraft'}, {'feature_label': 'Aircraft', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Aircraft')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Operator', 'transformation_label': 'String Indexer'}], 'feature': 'Operator', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': '363 Kingair', 'max': 'Zero One Tango', 'missing': '0', 'distinct': '123'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Operator'}, {'feature_label': 'Operator', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Operator')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Registration', 'transformation_label': 'String Indexer'}], 'feature': 'Registration', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '50099.24', 'stddev': '64463.8', 'min': '0685/4-G-1', 'max': 'ZS-DHX', 'missing': '0', 'distinct': '490'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Registration'}, {'feature_label': 'Registration', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Registration')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Flight phase', 'transformation_label': 'String Indexer'}], 'feature': 'Flight phase', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': 'Flight', 'max': 'Taxiing', 'missing': '0', 'distinct': '4'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Flight phase'}, {'feature_label': 'Flight phase', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Flight phase')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Flight type', 'transformation_label': 'String Indexer'}], 'feature': 'Flight type', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': 'Ambulance', 'max': 'Training', 'missing': '0', 'distinct': '23'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Flight type'}, {'feature_label': 'Flight type', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Flight type')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Survivors', 'transformation_label': 'String Indexer'}], 'feature': 'Survivors', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': 'NA', 'max': 'Yes', 'missing': '0', 'distinct': '3'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Survivors'}, {'feature_label': 'Survivors', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Survivors')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Crash site', 'transformation_label': 'String Indexer'}], 'feature': 'Crash site', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': 'Airport (less than 10 km from airport)', 'max': 'Plain, Valley', 'missing': '0', 'distinct': '7'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Crash site'}, {'feature_label': 'Crash site', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Crash site')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'MSN', 'transformation_label': 'String Indexer'}], 'feature': 'MSN', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '95929055.84', 'stddev': '1354109049.6', 'min': '00 347 607', 'max': 'UC-135', 'missing': '0', 'distinct': '474'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'MSN'}, {'feature_label': 'MSN', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('MSN')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'YOM', 'transformation_label': 'String Indexer'}], 'feature': 'YOM', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '1919.92', 'stddev': '1.06', 'min': '1918', 'max': 'NA', 'missing': '0', 'distinct': '48'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'YOM'}, {'feature_label': 'YOM', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('YOM')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Country', 'transformation_label': 'String Indexer'}], 'feature': 'Country', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': 'Afghanistan', 'max': 'Yemen', 'missing': '0', 'distinct': '42'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Country'}, {'feature_label': 'Country', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Country')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Region', 'transformation_label': 'String Indexer'}], 'feature': 'Region', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': 'Africa', 'max': 'World', 'missing': '0', 'distinct': '8'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Region'}, {'feature_label': 'Region', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Region')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Crew on board', 'transformation_label': 'String Indexer'}], 'feature': 'Crew on board', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '2.68', 'stddev': '2.53', 'min': '0', 'max': 'NA', 'missing': '0', 'distinct': '16'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Crew on board'}, {'feature_label': 'Crew on board', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Crew on board')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Crew fatalities', 'transformation_label': 'String Indexer'}], 'feature': 'Crew fatalities', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '0.51', 'stddev': '1.11', 'min': '0', 'max': '5', 'missing': '0', 'distinct': '7'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Crew fatalities'}, {'feature_label': 'Crew fatalities', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Crew fatalities')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Pax on board', 'transformation_label': 'String Indexer'}], 'feature': 'Pax on board', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '14.32', 'stddev': '36.08', 'min': '0', 'max': 'NA', 'missing': '0', 'distinct': '11'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Pax on board'}, {'feature_label': 'Pax on board', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Pax on board')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'PAX fatalities', 'transformation_label': 'String Indexer'}], 'feature': 'PAX fatalities', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '5.45', 'stddev': '18.67', 'min': '0', 'max': '99', 'missing': '0', 'distinct': '45'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'PAX fatalities'}, {'feature_label': 'PAX fatalities', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('PAX fatalities')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Other fatalities', 'transformation_label': 'String Indexer'}], 'feature': 'Other fatalities', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '0.01', 'stddev': '0.22', 'min': '0', 'max': '5', 'missing': '0', 'distinct': '2'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Other fatalities'}, {'feature_label': 'Other fatalities', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Other fatalities')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Crash cause', 'transformation_label': 'String Indexer'}], 'feature': 'Crash cause', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': 'Human factor', 'max': 'Weather', 'missing': '0', 'distinct': '6'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Crash cause'}, {'feature_label': 'Crash cause', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Crash cause')
        display(transformationDF.limit(2).toPandas())
        return transformationDF


***AUTOML FUNCTIONS***

In [ ]:
from tpot import TPOTClassifier
from sklearn.model_selection import train_test_split
import pyspark


def functionClassification(sparkDF, listOfFeatures, label):
    sparkDF.persist(pyspark.StorageLevel.MEMORY_AND_DISK)
    df = (sparkDF.toPandas())
    X = (df.drop(label, axis=1))[listOfFeatures].values
    y = df[label].values
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, random_state=1, test_size=0.1)
    tpotModel = TPOTClassifier(verbosity=3, n_jobs=-1, generations=10, max_time_mins=5,
                               population_size=15, use_dask=True)
    tpotModel.fit(X_train, y_train)
    display(" Accuracy of Model : %s" % tpotModel.score(X_test, y_test))
    data = {'model': tpotModel,
            'X_test': X_test,
            'y_test': y_test,
            'label': label,
            'columnNames': listOfFeatures}
    return data


***READING DATAFRAME***

In [ ]:
############## CREATE SPARK SESSION ############################ ENTER YOUR SPARK MASTER IP AND PORT TO CONNECT TO SERVER ################
from pyspark.sql import SparkSession
spark = SparkSession.builder.master('local[1]').getOrCreate()
#%run crashesclassifyHooks.ipynb
try:
	#sourcePreExecutionHook()

	planecrashes = HDFSConnector.fetch(spark, "{'url': '/FileStore/platform/uploadedSourceFiles/Plane Crashes.csv', 'filename': 'Plane Crashes.csv', 'delimiter': ',', 'file_type': 'Delimeted', 'is_header': 'Use Header Line', 'domain': 'http://172.31.59.158', 'port': '40070', 'dirPath': '/FileStore/platform', 'server_url': '/nexusMax/NexusMaxPlatform/uploads/platform/'}")
	#sourcePostExecutionHook(planecrashes)

except Exception as ex: 
	logging.error(ex)
#spark.stop()


***TRANSFORMING DATAFRAME***

In [ ]:
#%run crashesclassifyHooks.ipynb
try:
	#transformationPreExecutionHook()

	crashesclassifyautofe = TransformationMain.run(planecrashes,json.dumps( {"FE": [{"transformationsData": [{"feature_label": "Date", "transformation_label": "String Indexer"}], "feature": "Date", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "1/11/1992", "max": "9/9/1991", "missing": "0", "distinct": "414"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Date"}, {"transformationsData": [{"feature_label": "Aircraft", "transformation_label": "String Indexer"}], "feature": "Aircraft", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "Aeritalia G.222", "max": "Yakovlev Yak-42", "missing": "0", "distinct": "162"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Aircraft"}, {"transformationsData": [{"feature_label": "Operator", "transformation_label": "String Indexer"}], "feature": "Operator", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "363 Kingair", "max": "Zero One Tango", "missing": "0", "distinct": "123"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Operator"}, {"transformationsData": [{"feature_label": "Registration", "transformation_label": "String Indexer"}], "feature": "Registration", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "50099.24", "stddev": "64463.8", "min": "0685/4-G-1", "max": "ZS-DHX", "missing": "0", "distinct": "490"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Registration"}, {"transformationsData": [{"feature_label": "Flight phase", "transformation_label": "String Indexer"}], "feature": "Flight phase", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "Flight", "max": "Taxiing", "missing": "0", "distinct": "4"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Flight phase"}, {"transformationsData": [{"feature_label": "Flight type", "transformation_label": "String Indexer"}], "feature": "Flight type", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "Ambulance", "max": "Training", "missing": "0", "distinct": "23"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Flight type"}, {"transformationsData": [{"feature_label": "Survivors", "transformation_label": "String Indexer"}], "feature": "Survivors", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "NA", "max": "Yes", "missing": "0", "distinct": "3"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Survivors"}, {"transformationsData": [{"feature_label": "Crash site", "transformation_label": "String Indexer"}], "feature": "Crash site", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "Airport (less than 10 km from airport)", "max": "Plain, Valley", "missing": "0", "distinct": "7"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Crash site"}, {"transformationsData": [{"feature_label": "MSN", "transformation_label": "String Indexer"}], "feature": "MSN", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "95929055.84", "stddev": "1354109049.6", "min": "00 347 607", "max": "UC-135", "missing": "0", "distinct": "474"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "MSN"}, {"transformationsData": [{"feature_label": "YOM", "transformation_label": "String Indexer"}], "feature": "YOM", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "1919.92", "stddev": "1.06", "min": "1918", "max": "NA", "missing": "0", "distinct": "48"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "YOM"}, {"transformationsData": [{"feature_label": "Country", "transformation_label": "String Indexer"}], "feature": "Country", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "Afghanistan", "max": "Yemen", "missing": "0", "distinct": "42"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Country"}, {"transformationsData": [{"feature_label": "Region", "transformation_label": "String Indexer"}], "feature": "Region", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "Africa", "max": "World", "missing": "0", "distinct": "8"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Region"}, {"transformationsData": [{"feature_label": "Crew on board", "transformation_label": "String Indexer"}], "feature": "Crew on board", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "2.68", "stddev": "2.53", "min": "0", "max": "NA", "missing": "0", "distinct": "16"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Crew on board"}, {"transformationsData": [{"feature_label": "Crew fatalities", "transformation_label": "String Indexer"}], "feature": "Crew fatalities", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "0.51", "stddev": "1.11", "min": "0", "max": "5", "missing": "0", "distinct": "7"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Crew fatalities"}, {"transformationsData": [{"feature_label": "Pax on board", "transformation_label": "String Indexer"}], "feature": "Pax on board", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "14.32", "stddev": "36.08", "min": "0", "max": "NA", "missing": "0", "distinct": "11"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Pax on board"}, {"transformationsData": [{"feature_label": "PAX fatalities", "transformation_label": "String Indexer"}], "feature": "PAX fatalities", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "5.45", "stddev": "18.67", "min": "0", "max": "99", "missing": "0", "distinct": "45"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "PAX fatalities"}, {"transformationsData": [{"feature_label": "Other fatalities", "transformation_label": "String Indexer"}], "feature": "Other fatalities", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "0.01", "stddev": "0.22", "min": "0", "max": "5", "missing": "0", "distinct": "2"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Other fatalities"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Total fatalities", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "0.73", "stddev": "1.61", "min": "0", "max": "17", "missing": "0"}, "updatedLabel": "Total fatalities"}, {"transformationsData": [{"feature_label": "Crash cause", "transformation_label": "String Indexer"}], "feature": "Crash cause", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "Human factor", "max": "Weather", "missing": "0", "distinct": "6"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Crash cause"}]}))

	#transformationPostExecutionHook(crashesclassifyautofe)

except Exception as ex: 
	logging.error(ex)


***TRAIN MODEL***

In [ ]:
#%run crashesclassifyHooks.ipynb
try:
	#mlPreExecutionHook()

	dataAutoML=functionClassification(crashesclassifyautofe, ["Total fatalities", "Date_stringindexer", "Aircraft_stringindexer", "Operator_stringindexer", "Registration_stringindexer", "Flight phase_stringindexer", "Flight type_stringindexer", "Crash site_stringindexer", "MSN_stringindexer", "YOM_stringindexer", "Country_stringindexer", "Region_stringindexer", "Crew on board_stringindexer", "Crew fatalities_stringindexer", "Pax on board_stringindexer", "PAX fatalities_stringindexer", "Other fatalities_stringindexer", "Crash cause_stringindexer"], "Survivors_stringindexer")

	#mlPostExecutionHook(dataAutoML)

except Exception as ex: 
	logging.error(ex)
#spark.stop()


***PREDICT ON TRAINED MODEL***

In [ ]:
import pandas as pd
import numpy as np
import sklearn.metrics

try:
    model=dataAutoML['model']
    X_test=dataAutoML['X_test']
    y_test=dataAutoML['y_test']
    label=dataAutoML['label']
    columnNames=dataAutoML['columnNames']
    if label in columnNames:
        columnNames.remove(label)
    predicted=label+"_predicted"
    y_predicted=model.predict(X_test)
    df =pd.DataFrame(X_test , columns=columnNames)
    df[label]=y_test
    df[predicted]=y_predicted
    columnNames.insert(0,predicted)
    columnNames.insert(0,label)
    Accuracy = np.round((100 * sklearn.metrics.accuracy_score(y_true=y_test, y_pred=y_predicted)), 1)
    F1= np.round(
            (100 * sklearn.metrics.f1_score(y_true=y_test, y_pred=y_predicted, average="weighted")), 1)
    Precision= np.round((
                100 * sklearn.metrics.precision_score(y_true=y_test, y_pred=y_predicted, average="weighted")), 1)
    Recall = np.round((
                100 * sklearn.metrics.recall_score(y_true=y_test, y_pred=y_predicted, average="weighted")), 1)
    display(" Accuracy of Prediction on test data    : %s"%Accuracy)
    display(" F1 score of Prediction on test data    : %s"%F1)
    display(" Precision of Prediction on test data   : %s"%Precision)
    display(" Recall of Prediction on test data      : %s"%Recall)
    display(df.head())
except Exception as ex:
    logging.error(ex)

spark.stop()

